# Korean Hate Speech Classifier
## Written by: [Jehwan Kim](github.com/kreimben)
## Date: 19th Feb 2024
## Referenced Paper: 
* [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882)
* [Cyclical Learning Rates for Training Neural Networks](https://arxiv.org/abs/1506.01186)

### Load the word2vec model first,

In [ ]:
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


%matplotlib inline

### And then, load labeled data using pandas

In [ ]:
dev_df = pd.read_csv('./labeled/dev.tsv', sep='\t')
train_df = pd.read_csv('./labeled/train.tsv', sep='\t')

In [ ]:
train_df.head()

In [ ]:
dev_df.head()

### In `hate` column, `offensive`, `none`, `hate`.
### In `contain_gender_bias` column, `True`, `False`.
### In `bias` column, `none`, `gender`, `others`.

In [ ]:
# combine train and dev data.
df = pd.concat([dev_df, train_df], ignore_index=True)
df.sample(5)

In [ ]:
df.hate.value_counts(), dev_df.contain_gender_bias.value_counts(), dev_df.bias.value_counts()

### Load words data and tokeniser from past project.

In [ ]:
import pickle

with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('words.pkl', 'rb') as handle:
    words = pickle.load(handle)

In [ ]:



def tokenize(sentence):
    tokens = tokenizer.tokenize(sentence)
    preprocess = lambda x: [w for w in x if w not in STOP_WORDS]
    return preprocess(tokens)

In [ ]:
sample = df.sample(1).comments.values[0]

sample, tokenize(sample)

In [ ]:
df['tokens'] = df['comments'].apply(tokenize)
# 공격적인(offensive) 댓글 또한 혐오 데이터 셋으로 분류함.
df['hate'] = df['hate'].replace(['none', 'offensive', 'hate'], [0, 1, 1])
df['contain_gender_bias'] = df['contain_gender_bias'].replace([True, False], [1, 0])
df.head()

In [ ]:
vocab = set()
for sentence in df.tokens:
    for word in sentence: vocab.add(word)

vocab_size = len(vocab)

### Vectorise

In [ ]:
with open('vectorizer.pkl', 'rb') as handle:
    vectorizer = pickle.load(handle)

### Encoding

In [ ]:
df['encoding'] = df['comments'].apply(vectorizer.encode_a_doc_to_list)
df.encoding

In [ ]:
X_data = df['encoding']
y_data = df['hate']
print(y_data.value_counts())
len(X_data), len(y_data), len(X_data) == len(y_data)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=.1, random_state=0, stratify=y_data)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.1, random_state=0, stratify=y_train)

y_train.value_counts()

In [ ]:
print('공격적인(offensive) 댓글 또한 혐오 데이터 셋으로 분류함.')
print('--------훈련 데이터의 비율-----------')
print(f'혐오 댓글 = {round(y_train.value_counts()[1] / len(y_train) * 100, 3)}%')
print(f'일반 댓글 = {round(y_train.value_counts()[0] / len(y_train) * 100, 3)}%')
print('--------검증 데이터의 비율-----------')
print(f'혐오 댓글 = {round(y_valid.value_counts()[1] / len(y_valid) * 100, 3)}%')
print(f'일반 댓글 = {round(y_valid.value_counts()[0] / len(y_valid) * 100, 3)}%')
print('--------테스트 데이터의 비율-----------')
print(f'혐오 댓글 = {round(y_test.value_counts()[1] / len(y_test) * 100, 3)}%')
print(f'일반 댓글 = {round(y_test.value_counts()[0] / len(y_test) * 100, 3)}%')

### Padding

In [ ]:
print('댓글의 최대 길이 :', max(len(review) for review in X_train))
print('댓글의 평균 길이 :', sum(map(len, X_train)) / len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
max_len = max(len(review) for review in X_train)

### 길이가 길지 않아서 그냥 최대값 그대로 진행합니다. (패딩)

In [ ]:
def pad_sequences(sentences: [[int]], max_len: int) -> np.ndarray:
    features = np.zeros((len(sentences), max_len), dtype=int)
    for index, sentence in enumerate(sentences):
        if len(sentence) != 0:
            features[index, :len(sentence)] = np.array(sentence)[:max_len]
    return features

In [ ]:
padded_X_train = pad_sequences(X_train, max_len=max_len)
padded_X_valid = pad_sequences(X_valid, max_len=max_len)
padded_X_test = pad_sequences(X_test, max_len=max_len)

print('훈련 데이터의 크기 :', padded_X_train.shape)
print('검증 데이터의 크기 :', padded_X_valid.shape)
print('테스트 데이터의 크기 :', padded_X_test.shape)

In [ ]:
padded_X_test[:5, :]

### Modeling

In [ ]:
train_label_tensor = torch.tensor(np.array(y_train))
valid_label_tensor = torch.tensor(np.array(y_valid))
test_label_tensor = torch.tensor(np.array(y_test))
print(train_label_tensor[:5])

In [ ]:
class TextCNNLightning(L.LightningModule):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, train_batch_size):
        super().__init__()
        self.lr = None
        self.train_batch_size = train_batch_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [batch size, sent len]
        embedded = self.embedding(text)  # embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)  # embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        # conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim=1))
        # cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def training_step(self, batch, batch_idx):
        # Get inputs and labels
        inputs, labels = batch

        # Forward pass
        outputs = self(inputs)

        # Calculate loss
        loss = F.cross_entropy(outputs, labels)

        # Log loss
        self.log("train_loss", loss)

        # Return loss
        return loss

    def __accuracy(self, outputs, labels):
        predictions = outputs.argmax(dim=1)  # Get indices of highest probability
        correct = (predictions == labels).sum().item()
        acc = correct / len(labels)
        return acc

    def validation_step(self, batch, batch_idx):
        # Get inputs and labels
        inputs, labels = batch

        # Forward pass
        outputs = self(inputs)

        # Calculate loss
        loss = F.cross_entropy(outputs, labels)

        # Calculate accuracy
        acc = self.__accuracy(outputs, labels)

        # Log loss and accuracy
        self.log("val_loss", loss)
        self.log("val_acc", acc)

        # Return loss and accuracy
        return loss, acc

    def test_step(self, batch, batch_idx):
        # Get inputs and labels
        inputs, labels = batch

        # Forward pass
        outputs = self(inputs)

        # Calculate loss
        loss = F.cross_entropy(outputs, labels)

        # Calculate accuracy
        acc = self.__accuracy(outputs, labels)

        # Log loss and accuracy
        self.log("test_loss", loss)
        self.log("test_acc", acc)

        # Return loss and accuracy
        return loss, acc

    def train_dataloader(self):
        encoded_train = torch.tensor(padded_X_train).to(torch.int32)
        train_dataset = TensorDataset(encoded_train, train_label_tensor)
        train_dataloader = DataLoader(train_dataset, shuffle=True, num_workers=7,
                                      persistent_workers=True, batch_size=self.train_batch_size)
        return train_dataloader

    def val_dataloader(self):
        encoded_valid = torch.tensor(padded_X_valid).to(torch.int32)
        valid_dataset = TensorDataset(encoded_valid, valid_label_tensor)
        valid_dataloader = DataLoader(valid_dataset, shuffle=False, batch_size=1, num_workers=7,
                                      persistent_workers=True)
        return valid_dataloader

    def test_dataloader(self):
        encoded_test = torch.tensor(padded_X_test).to(torch.int32)
        test_dataset = TensorDataset(encoded_test, test_label_tensor)
        test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1, num_workers=7)
        return test_dataloader

In [ ]:
# # HPO using optuna
# 
# def objective(trial):
#     # Define the hyperparameter space
#     embedding_dim = trial.suggest_int("embedding_dim", 100, 500)
#     n_filters = trial.suggest_int("n_filters", 100, 300)
#     dropout_rate = trial.suggest_float('dropout_rate', .1, .9)
# 
#     # Suggest a logarithmic value
#     log_base_2_value = trial.suggest_int('log_base_2_value', 0, 10)
#     # Convert to actual value
#     train_batch_size = 2 ** log_base_2_value
# 
#     # Initialize the model with the hyperparameters
#     model = TextCNNLightning(vocab_size=vocab_size, embedding_dim=embedding_dim, n_filters=n_filters,
#                              filter_sizes=[3, 4, 5], output_dim=2, dropout=dropout_rate,
#                              train_batch_size=train_batch_size)
# 
#     # Trainer settings
#     trainer = L.Trainer(
#         accelerator="auto", max_epochs=15
#     )
# 
#     # Train the model
#     trainer.fit(model)
# 
#     # Evaluate the model performance
#     return trainer.callback_metrics["train_loss"]


In [ ]:
# import optuna
# 
# # Create a study object
# study = optuna.create_study()  # or 'maximize' based on your goal
# study.optimize(objective, n_trials=100)  # Specify the number of trials
# 
# # Print the best hyperparameters
# print(f"Best trial: {study.best_trial.params}")


In [ ]:
# model = TextCNNLightning(vocab_size=vocab_size, num_labels=len(set(y_train)), l=l)
model = TextCNNLightning(vocab_size=vocab_size, embedding_dim=300, n_filters=100,
                         filter_sizes=[3, 4, 5], output_dim=2, dropout=.5, train_batch_size=512)

vocab_size, len(set(y_train))

### Train

In [ ]:
%%time

from lightning.pytorch.callbacks import LearningRateFinder

trainer = L.Trainer(
    accelerator="auto", devices="auto", strategy="auto",
    max_epochs=10, callbacks=[LearningRateFinder()]
)
trainer.fit(model=model)

In [ ]:
trainer.test()

### Using Bert to classificate

In [ ]:
from icecream import ic
from utils.evaluate import calculate_accuracy
from transformers import BertModel


class BertClassifier(L.LightningModule):
    def __init__(self, n_classes: int, pretrained_model_name="klue/bert-base", steps_per_epoch=None, n_epochs=None, batch_size=32):
        super().__init__()
        self.batch_size = batch_size
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return self.classifier(output.pooler_output)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        
        input_ids = input_ids.reshape(-1, input_ids.shape[-1])  # Keep last dimension dynamic
        attention_mask = attention_mask.reshape(-1, attention_mask.shape[-1])
        
        outputs = self(input_ids, attention_mask)
        loss = self.criterion(outputs, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Get inputs and labels
        input_ids, attention_mask, labels = batch
        
        input_ids = input_ids.reshape(-1, input_ids.shape[-1])  # Keep last dimension dynamic
        attention_mask = attention_mask.reshape(-1, attention_mask.shape[-1])

        # Forward pass
        outputs = self(input_ids, attention_mask)

        # Calculate loss
        loss = F.cross_entropy(outputs, labels)

        # Calculate accuracy
        acc = calculate_accuracy(outputs, labels)

        # Log loss and accuracy
        self.log("val_loss", loss)
        self.log("val_acc", acc)

        # Return loss and accuracy
        return loss, acc

    def test_step(self, batch, batch_idx):
        # Get inputs and labels
        input_ids, attention_mask, labels = batch

        # Forward pass
        outputs = self(input_ids, attention_mask)

        # Calculate loss
        loss = F.cross_entropy(outputs, labels)

        # Calculate accuracy
        acc = calculate_accuracy(outputs, labels)

        # Log loss and accuracy
        self.log("test_loss", loss)
        self.log("test_acc", acc)

        # Return loss and accuracy
        return loss, acc

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=self.steps_per_epoch * 0.1, gamma=0.1)
        return [optimizer], [scheduler]

In [ ]:
from sklearn.model_selection import train_test_split
from utils.stopwords import STOP_WORDS

dev_df = pd.read_csv('./labeled/dev.tsv', sep='\t')
train_df = pd.read_csv('./labeled/train.tsv', sep='\t')
df = pd.concat([dev_df, train_df], ignore_index=True)

from torch.utils.data import random_split
from transformers import BertTokenizer

bert_model = BertModel.from_pretrained("klue/bert-base")
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

tokenized_batch = tokenizer.batch_encode_plus(df.comments.values, padding=True, truncation=True, return_tensors='pt')

input_ids = tokenized_batch['input_ids']
attention_mask = tokenized_batch['attention_mask']
y_label_tensor = torch.tensor(np.array(y_data))

dataset = TensorDataset(input_ids, attention_mask, y_label_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])
train_dataloader, val_dataloader, test_dataloader = DataLoader(train_set, batch_size=32, shuffle=True), DataLoader(val_set, batch_size=32), DataLoader(test_set, batch_size=32)

vocab_size = tokenizer.vocab_size

In [ ]:
# 모델 인스턴스화 및 훈련 파라미터 설정
bert_classifier_model = BertClassifier(n_classes=2, steps_per_epoch=100, n_epochs=3)

# 트레이너 설정 및 훈련 시작
bert_trainer = L.Trainer(max_epochs=3)
bert_trainer.fit(bert_classifier_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

In [ ]:
bert_trainer.test(dataloaders=[test_dataloader])

# 최종 결과

* 이전 프로젝트인 word2vec에서 직접 가져온 tokeniser로 단어 임베딩을 진행한 결과 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6463560461997986
        test_loss           0.7093809247016907
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
정확도는 64%이고 손실률은 0.7이 됐다.
손수 직접 임베딩을 한 것이라, 아무리 epoch를 더 돌린다 해도 정확도가 더 올라가지 않아 포기했다.
HPO 관련 라이브러리를 설치해봤으나 시간이 너무 부족해 hpo는 하지 못했다.

* bert에서 가져온 tokeniser로 단어 임베딜을 진행한 결과
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7778662443161011
        test_loss           0.7982401847839355
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
정확도는 77%이고 손실률은 0.79이다.

---

프로젝트를 진행하면서 의문이였던 점은, 'tokeniser의 차이로 문장들의 전체 길이가 달라진다는 점이 최종 결과까지 영향을 줄 수 있는가'이다.
또한 '이모티콘이나 자음을 이용해 ㅇㅁㅇ 이나 (ㅇㅅㅇ) 와 같은 것을 쓴다면 이 역시 stopword나 정규식을 이용해 필터링 해야하는가'이다.
한국인들은 ㅋ 와 ㅋㅋ 의 의미를 다르게 받아들인다. 이럴 경우 tokenising 처리를 어디까지 해줘야 하는지가 관건인것 같다. 